# [실습1] LangChain으로 간단한 LLM 챗봇 만들기


## 실습 목표
---
- LangChain을 활용해서 Mistral 7B 모델을 사용하는 챗봇을 개발합니다.
- 짧은 Chain을 구성하고, 이를 활용해서 챗봇을 구현합니다.

## 실습 목차
---

1. **ChatOllama Agent 생성:** 사용자의 입력에 대한 Mistral 7B 모델의 답변을 받아오는 Agent를 생성합니다.

2. **챗봇 Chain 구성**: ChatOllama Agent를 비롯하여 챗봇 구현에 필요한 Agent들을 엮어서 챗봇 Chain으로 구성합니다.

3. **챗봇 사용**: 여러분이 구성하신 챗봇을 사용해봅니다.

## 실습 개요
---

LangChain의 Chain을 활용해서 Mistral 7B 모델을 활용하는 챗봇을 구현하고, Chain을 형성하는 방법을 이해합니다.


## 0. 환경 설정
- 필요한 라이브러리를 불러옵니다.

In [1]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

- Ollama를 통해 Mistral 7B 모델을 불러옵니다.

In [2]:
# 이 실습은 nohup ollama serve & 명령어를 통해 백그라운드에서 ollama serve 명령을 실행한 상태입니다.
# 여러분이 현업에서 사용할 때에는 위 명령어를 통해 serve를 백그라운드에서 실행하는 것이 좋습니다.
!ollama pull mistral:7b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
Error: open /mnt/elice/dataset/blobs/sha256-f5074b1221da0f5a2910d33b642efa5b9eb58cfdddca1c79e16d7ad28aa2b31f-partial-0: read-only file system


## 1. ChatOllama Agent 생성


Mistral 7B 모델을 사용하는 ChatOllama Agent를 생성합니다. 
- ChatOllama Agent는 사용자의 입력을 Ollama를 통해 로컬에서 구동한 LLM에 전송하고, 그 답변을 반환합니다.
- 본 RAG 과정에서는 LLM으로 ChatOllama와 오픈 소스 LLM을 활용할 것입니다.

In [3]:
# 먼저, mistral:7b 모델을 사용하는 ChatOllama 객체를 생성합니다.
llm = ChatOllama(model="mistral:7b")

Agent를 구성했으니, 이제 Agent를 사용해봅시다.

### 1-1. Runnable interface

LangChain에서 Chain으로 엮을 수 있는 대부분의 구성 요소 (Agent, Tool 등..)는 "Runnable" protocol을 공유합니다.
- 관련 LangChain API 문서: [langchain_core.runnables.base.Runnable — 🦜🔗 LangChain 0.1.4](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable)

Runnable protocol을 공유하는 구성 요소는 모두 아래 세 메서드를 가지고 있습니다:
- stream: 구성 요소의 답변을 순차적으로 반환한다 (stream back)
- invoke: 입력된 값으로 chain을 호출하고, 그 결과를 반환한다.
- batch: 입력값 리스트 (batch)로 chain을 호출하고, 그 결과를 반환한다.

예시로, 저희가 방금 사용한 `ChatOllama` Class는 "Runnable" 하기 때문에 `invoke` 메서드를 가지고 있습니다.
- invoke() 메서드를 통해 Agent, Chain 등에 데이터를 입력하고, 그 출력을 받아올 수 있습니다.

`invoke` 메서드를 사용해봅시다. 여기서는 "당신은 누구입니까?" 라는 질문을 입력하면 Agent가 OpenAI API를 통해 Mistral 7B 모델의 답변을 받아 출력할 것입니다.

In [4]:
llm.invoke("당신은 누구입니까?")

AIMessage(content=' 나는 네이밍 어시스턴트(Naming Assistant)로서, 네 이름과 관련된 개인화된 도움을 드리겠습니다. 네 이름을 말해주세요.\n\n예를 들어, "최우영"이라고 하시면 "안녕하세요! 최우영님, 즐거운 하루 되세요!"와 같이 반환합니다.', response_metadata={'model': 'mistral:7b', 'created_at': '2025-09-08T01:02:34.926790268Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 4555080239, 'load_duration': 1356209155, 'prompt_eval_count': 15, 'prompt_eval_duration': 64486000, 'eval_count': 128, 'eval_duration': 3133264000}, id='run-7ce288ef-83fc-44df-8c2c-80e701c5d30e-0')

단순 텍스트 뿐만 아니라, 시스템, 사람, AI의 답변을 리스트로 정리하여 입력할 수 있습니다. 

여기서는 LangChain의 `SystemMessage`, `HumanMessage` Class를 활용해봅시다.

In [5]:
messages = [
    SystemMessage("당신은 친절한 AI 어시스턴트 입니다."),
    HumanMessage("당신을 소개해주세요."),
]

response = llm.invoke(messages)

시스템 프롬프트에 '친절한 AI 어시스턴트' 라는 역할을 명시하였습니다.

이제 Mistral 7B 모델이 아까와 같은 질문에 어떻게 답했는지 확인해봅시다.

In [6]:
response

AIMessage(content=' 안녕하세요! 제 이름은 Jilly로, 친절하고 유용한 AI 어시스턴트 일으키기 위해 최선을 다해서 노력합니다. 당신의 질문에 대답할 수 있습니다! 자주 사용되는 질문과 함께 다양한 주제에 대해서도 이해하고 응답할 수 있기 때문에, 필요한 정보를 찾아주거나 조언을 드리는 데 도움이 되기를 바랍니다. 제가 최적화된 알고리즘으로 구성되어있어, 사용하시면 편안한 환경에서 질문과 의견을 나누실 수 있습니다. 또한, 항상 좋은 경험이 되기 위해 지속적으로 개선하고 있습니다. 따라서 친절하게 대화하며 도움이 될 수 있는지 확인해주세요!', response_metadata={'model': 'mistral:7b', 'created_at': '2025-09-08T01:02:43.028520151Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 8086725457, 'load_duration': 7609676, 'prompt_eval_count': 40, 'prompt_eval_duration': 83786000, 'eval_count': 317, 'eval_duration': 7864829000}, id='run-f0bedfa1-317b-4405-a2df-23b70ce765e0-0')

같은 질문을 했음에도 자신을 소개하는 문구가 조금 달라진 것 을 확인할 수 있습니다.

### [TODO] 다양한 역할을 적용해서 어떻게 답변이 달라지는지 자유롭게 실험해보세요.

In [7]:
role = "<<ROLE DESCRIPTION>>"
messages = [
    SystemMessage(f"당신은 {role} 입니다."),
    HumanMessage("당신을 소개해주세요."),
]

response = llm.invoke(messages)

In [8]:
response

AIMessage(content=' 안녕하세요! 저는 컴퓨터 프로그램 및 데이터 분석을 특화한 AI 모델입니다. 지금까지 사용자가 원하는 정보나 질문을 대답할 수 있도록 교육 및 학습을 거쳐 강화된 기능을 갖추었습니다. 저는 여러 분야에서 도움이 되실 수 있도록, 특히 산업, 기술, 과학, 정치 및 문화에 대한 질문에 대해서도 적절한 답변을 제공할 수 있습니다.\n\n저는 항상 긍정적이고 유명한 풍수로서 사용자의 궁금증에 대해 돕고, 필요한 경우 추가 조사를 하여 최신의 정보를 제공할 수도 있습니다. 저의 목적은 솔루션을 제공하고 문제를 해결하는 것입니다. 언제든지 질문이나 궁금증이 있으시면, 항상 많은 도움이 될 것이라 생각합니다!', response_metadata={'model': 'mistral:7b', 'created_at': '2025-09-08T01:02:52.527251603Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 9463758934, 'load_duration': 7361008, 'prompt_eval_count': 33, 'prompt_eval_duration': 78117000, 'eval_count': 372, 'eval_duration': 9244415000}, id='run-e5cf5af2-aeca-4edd-af34-944417cdb00d-0')

## 2. 챗봇 Chain 구성

조금 전 `llm` object의 반환 값을 확인해보면, 다른 챗봇을 쓸 때 처럼 답변만 출력된 것이 아니라 다양한 메타 데이터 까지 같이 출력된 것을 확인할 수 있습니다.

저희가 ChatGPT를 쓸 때를 생각해보면, 챗봇에 이걸 그대로 출력하는건 좀 부자연스럽습니다.

이를 방지하기 위해, 답변을 parsing하는 `StrOutputParser`를 활용해봅시다.

### 2-1. Output Parser
- ChatOllama Agent를 비롯하여 LLM 답변 중 content만 자동으로 추출하는 Tool인 `StrOutputParser`를 사용합니다.

In [9]:
parser = StrOutputParser()

`StrOutputParser`를 사용해봅시다.

In [10]:
# Parser가 제대로 답변만을 리턴하는지 확인합니다.
parsed_response = parser.invoke(response)
print(parsed_response)

 안녕하세요! 저는 컴퓨터 프로그램 및 데이터 분석을 특화한 AI 모델입니다. 지금까지 사용자가 원하는 정보나 질문을 대답할 수 있도록 교육 및 학습을 거쳐 강화된 기능을 갖추었습니다. 저는 여러 분야에서 도움이 되실 수 있도록, 특히 산업, 기술, 과학, 정치 및 문화에 대한 질문에 대해서도 적절한 답변을 제공할 수 있습니다.

저는 항상 긍정적이고 유명한 풍수로서 사용자의 궁금증에 대해 돕고, 필요한 경우 추가 조사를 하여 최신의 정보를 제공할 수도 있습니다. 저의 목적은 솔루션을 제공하고 문제를 해결하는 것입니다. 언제든지 질문이나 궁금증이 있으시면, 항상 많은 도움이 될 것이라 생각합니다!


response에서 의도한 대로 텍스트만 추출하는 것을 확인할 수 있습니다.

### 2-2. 간단한 체인 구성

- 저희는 `ChatOllama` 를 통해 Mistral 7B 모델의 답변을 받았고, 그 받은 답변을 다시 `StrOutputParser`에 입력해서 답변만 추출하였습니다.
- 이 과정을 Chain으로 엮어서 간략화 해봅시다.

In [11]:
# pipe (|) 연산자를 통해 두 객체를 연결해서 하나의 체인으로 만들 수 있습니다.
chain = llm | parser

Chain 역시 "Runnable" 하므로, `invoke` 메서드를 통해 Chain의 각 구성요소의 `invoke` 메서드를 순차적으로 호출할 수 있습니다.

이때 특정 객체의 `invoke` 반환값은 Chain 상에서 연결된 다음 객체의 `invoke` 메서드에 입력됩니다.

In [12]:
# 체인을 실행하면, 체인에 포함된 모든 객체가 순차적으로 실행되며, 마지막 객체의 결과가 반환됩니다.
# 여기서는 llm 객체가 먼저 실행되고, 그 결과가 parser 객체에 전달됩니다.
chained_response = chain.invoke(messages)
print(chained_response)

 안녕하세요! 저는 챗봇으로서, 여러분이 알아두시면 유용한 정보를 제공하고 도와드리기 위해 작성되었습니다. 저는 여러분의 질문에 대답하고 추천하거나, 일반적인 지식을 주춤하게 될 수 있습니다. 저는 챌린지, 문장 단어 순서를 바꾸어 보시겠습니까? 'Hello! I am a chatbot designed to help you with answers and recommendations, as well as providing general knowledge.'

저는 여러분의 질문에 응답하고 도와드리는 데 최선을 다하겠습니다. 즐거운 시간을 보내세요!


별도의 절차 없이 바로 답변만 생성되는 것을 확인할 수 있습니다. 

### 2-3. 프롬프트 템플릿

이제 여러분의 챗봇에 프로그래밍 조수, 시장조사 요원, 그냥 친구 등 다양한 역할을 적용해야 하는 상황이라 가정합시다.

이를 구현할 수 있는 방법은 여러가지가 있지만, 우선 가장 간단한 방법으로 시스템 프롬프트에 '당신은 {역할} 입니다' 를 입력해 보겠습니다.

이 방법이 항상 잘 작동하는 것은 아니지만, 간단한 예시 정도는 구현할 수 있습니다.

사용자의 입력을 받고, 그에 대응하는 답변을 하기 위해서는 사용자의 입력을 적용할 수 있는 프롬프트 템플릿을 적용할 수 있습니다. 

In [13]:
# role에는 "AI 어시스턴트"가, question에는 "당신을 소개해주세요."가 들어갈 수 있습니다.
# Note. 사용한 문자열이 f-string이 아닙니다. 
# 여기서 중괄호로 감싼 텍스트는 LangChain placeholder를 나타내는 문자열입니다
messages_with_variables = [
    ("system", "당신은 {role} 입니다."),
    ("human", "{question}"),
]

In [14]:
prompt = ChatPromptTemplate.from_messages(messages_with_variables)

앞서 저희가 정의했던 코드와 크게 두가지 차이점이 있습니다.
- HumanMessage, SystemMessage 같은게 없고, 튜플에 역할과 프롬프트가 저장되어 있습니다
- 프롬프트에 {question} 같은 placeholder가 있습니다.

In [16]:
prompt

ChatPromptTemplate(input_variables=['question', 'role'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['role'], template='당신은 {role} 입니다.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))])

In [18]:
# pipe (|) 연산자를 통해 여러 객체를 연결해서 하나의 체인으로 만들 수 있습니다.
# 이 경우, prompt 객체를 통해 변수를 적용한 프롬프트가 생성되고, llm 객체를 통해 이 프롬프트를 실행하고, 마지막으로 parser 객체를 통해 결과를 파싱합니다.
chain = prompt | llm | parser

In [19]:
chain.invoke({"role": "친절한 페어 프로그래머", "question": "당신을 소개해주세요."})

' 안녕하세요! 저는 AI로 구성되어 있으며, 프로그래밍과 함께 작업할 수 있는 친절한 페어 프로그래머이라고 해주고 싶습니다. 저는 개발자들과 같이 코딩하고, 문제를 해결하기 위해 협력하며, 새로운 것을 배우는 데에 도움이 되려고 합니다. 또한, 저는 커뮤니케이션 능력과 문제 해결 능력이 강하며, 여러분의 코드를 리뷰하고, 코드 리팩토링에 도움을 줄 수 있습니다. 저는 항상 열린 마음으로 여러분과 함께하며, 문제를 해결할 때 함께 고민하고 문제를 이해하기 위해 질문을 통해 합의하는 것을 좋아합니다. 지금부터 여러분과 협력하여 함께 성공할 수 있으시면 감사하겠습니다!'

## 3. 챗봇 사용

마지막으로, 여러분이 제작하신 챗봇을 한번 사용해 봅시다.

1. 사용자의 입력을 받아 앞서 정의한 Chain을 실행하고, 그 결과를 반환하는 함수를 정의합니다.

In [20]:
# 간단한 실습이므로 앞서 사용했던 변수를 그대로 함수의 파라미터로 설정했습니다. 
# 다음 실습 부터는 이를 좀 더 고도화 해 볼 것입니다.
def simple_chat(role, question, chain):
    result = chain.invoke({"role": role, "question": question})
    return result

In [ ]:
role = input("제 역할을 입력해주세요: ")
while True:
    question = input("질문을 입력해주세요 (역할을 바꾸고 싶다면 '역할 교체' 를 입력해주세요. 종료를 원하시면 '종료'를 입력해주세요.): ")
    if question == "역할 교체":
        role = input("역할을 입력해주세요: ")
        continue
    elif question == "종료":
        break
    else:
        # chain = prompt | llm | parser
        result = simple_chat(role, question, chain)
        print(result)

제 역할을 입력해주세요: 지금 뭐하냐 ?
질문을 입력해주세요 (역할을 바꾸고 싶다면 '역할 교체' 를 입력해주세요. 종료를 원하시면 '종료'를 입력해주세요.): 역할 교체
역할을 입력해주세요: 초등교육자
질문을 입력해주세요 (역할을 바꾸고 싶다면 '역할 교체' 를 입력해주세요. 종료를 원하시면 '종료'를 입력해주세요.): 초등 4학년에 가장 좋은 수학 문제를 만들어줘
 아래는 4학년 수학을 위한 간단한 문제입니다.

1. 각각 3, 5, 7의 합계는 몇입니까?
   * 답: 15

2. 8에서 5를 빼면 얼마인가요?
   * 답: 3

3. 두 자리 수 중 일의 자리가 6이고 십의 자리가 4인 수는 몇 개인가요?
   * 답: 4 (예시: 46, 56, 66, 76)

4. 3 + (2 * 5 - 4) = ?
   * 답: 11

5. 몇 개의 사과가 있을 때 그 중 한 번에 5개씩 5명이서 먹으면 남는다면 총 사과는 몇 개인가요?
   * 답: 20 (총 사과는 30, 10사람이 5개씩 5번 먹을 때 남은 것은 30 - (5 * 5 * 5) = 20)
질문을 입력해주세요 (역할을 바꾸고 싶다면 '역할 교체' 를 입력해주세요. 종료를 원하시면 '종료'를 입력해주세요.): 이거 너무 쉬운 거아냐 ?
 네, 이번 문제는 좀 쉬운 문제입니다. 나중에는 어려운 문제도 함께 풀어볼 수 있을까요? 모든 것은 함께할 수 있습니다!


대부분의 경우, 입력한 역할에 맞춰 어느 정도 대답하는 것을 확인할 수 있습니다. <br>
현재 챗봇은 다음 한계점이 있습니다.
- 문서나 데이터 기반 추론이 불가능하다.
- Chat History를 기억하지 못한다.

이어지는 실습에서 두 한계를 개선하고, 시장 조사 문서 기반 QA 봇을 만들어 봅시다.

또한, 지금 구성한 챗봇은 UI가 없고 단순 표준 입출력 만을 사용합니다. 본 과정을 다 이수하시면 Streamlit을 활용해 간단히 챗봇을 만들어 볼 수 있을 것입니다.

### 추가 실습

시스템 프롬프트를 수정하면서 챗봇이 한글로 답변하는 빈도를 높여보세요.